## Практическое задание


    1. Попробуйте изменить параметры нейронной сети работающей с датасетом imdb либо нейронной сети работающей c airline-passengers(она прилагается вместе с датасетом к уроку в виде отдельного скрипта) так, чтобы улучшить ее точность. Приложите анализ.
    2. Попробуйте изменить параметры нейронной сети генерирующий текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившейся у вас текст и опишите, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения.
    3.* Попробуйте на numpy реализовать нейронную сеть архитектуры LSTM
    4.* Предложите свои варианты решения проблемы исчезающего градиента в RNN

In [34]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.utils import pad_sequences

max_features = 20000
# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 80
batch_size = 65 # размер батча увеличен

In [29]:
print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

print('Pad последовательности (примеров в x единицу времени)')
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Построение модели...')
model = Sequential()
model.add(Embedding(max_features, 135))
model.add(LSTM(135, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности
Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Построение модели...
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 120)         2400000   
                                                                 
 lstm_6 (LSTM)               (None, 120)               115680    
                                                                 
 dense_6 (Dense)             (None, 1)                 121       
                                                                 
Total params: 2,515,801
Trainable params: 2,515,801
Non-trainable params: 0
_________________________________________________________________


In [51]:
from keras.optimizers import Adam,Adamax,RMSprop, Adagrad

In [53]:
# стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров 
model.compile(loss='binary_crossentropy',
              optimizer=Adagrad(learning_rate=0.01), #выбор оптимизатора Adagrad с ручной настройкой скорости обучения
              metrics=['accuracy'])

print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)

# Результат при тестировании: 0.3440701961517334
# Тестовая точность: 0.8518000245094299

Процесс обучения...
385/385 [==============================] - 29s 75ms/step - loss: 0.3441 - accuracy: 0.8518
Результат при тестировании: 0.3440701961517334
Тестовая точность: 0.8518000245094299


*В целом наиболее значимое улучшение вызвал выбор оптимизатора Adagrad. Увеличение количество слоев сети либо эпох не вызывает учучшения*

In [58]:
import numpy as np
from keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU
from keras.models import Sequential

In [59]:
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("utf-8", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

In [91]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID и определенный символом. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представления входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выбирается фиксированную длина последовательности 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 10
NUM_PREDS_PER_EPOCH = 100

In [92]:
model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer=RMSprop())


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
632/632 [==============================] - 794s 1s/step - loss: 2.5270
Генерация из посева: ary archive foundation and how your efforts and donations can help, see sections 3 and 4 and the foundation web page at 
ary archive foundation and how your efforts and donations can help, see sections 3 and 4 and the foundation web page at and the said the said the said the said the said the said the said the said the said the said the sa


Увеличение количества эпох и итераций приводит к генерации более осмысленного текста
to alice was said the groject gutenter the cousted - лучший пример

- Одним из вариантов решения проблемы исчезающего градиента является применение остаточный сетей (ResNets) - они создают ансамбль из множество мелких сетей. Остаточные нейронные сети достигают решения проблемы исчезающего градиента с помощью коротких путей или “пропущенных соединений” для перемещения по различным слоям. Пропуск слоев освобождает сеть от осложнений, упрощая ее, используя очень мало слоев на начальном этапе обучения. Это ускоряет обучение в десять раз, минимизируя эффект исчезновения градиентов.


- Можно использовать другие способы оптимизации, не использующие градиент (например генетический алгоритм)